In [ ]:
import fiona
import pyproj
from shapely.geometry import shape
from shapely.ops import transform

wgs84 = pyproj.CRS('EPSG:4326')
utm = pyproj.CRS('EPSG:32630')

project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform

with fiona.open("cutter.geojson", "r") as bbox_geojson:
    shapes = [transform(project, shape(feature["geometry"])) for feature in bbox_geojson]

print(shapes[0].wkt)

In [ ]:
import rasterio
from rasterio.mask import mask

LTS_PATH = "ECOv002_L2T_LSTE_26947_002_30TVK_20230407T112014_0710_01_LST.tif"

with rasterio.open(LTS_PATH) as src:
    mask_image, mask_transform = mask(src,
                                      shapes,
                                      crop=True,
                                      all_touched=True,
                                      nodata=src.profile["nodata"],)
    out_meta = src.meta

    out_meta.update({"driver": "GTiff",
                     "height": mask_image.shape[1],
                     "width": mask_image.shape[2],
                     "transform": mask_transform,
                     "nodata": src.profile["nodata"]
                     })

cutted_path = f"{LTS_PATH}.cut.tif"
with rasterio.open(cutted_path, "w", **out_meta) as dest:
        dest.write(mask_image)

In [ ]:
import numpy as np

with rasterio.open(cutted_path) as src:

    with fiona.open("house.geojson", "r") as house_geojson:
        house = [transform(project, shape(feature["geometry"])) for feature in house_geojson]

        # calculate the row and col of the house
        pixel_size = src.res[0]
        col = int((house[0].x - src.bounds.left) / pixel_size)
        row = int((src.bounds.top - house[0].y) / pixel_size)

        lts = src.read(1)
        lts[np.isnan(lts)] = 0
        max = np.amax(lts)
        min = np.amin(lts[np.nonzero(lts)])
        mean = np.mean(lts[np.nonzero(lts)])
        value = lts[row, col]

        to_degrees = lambda kelvin: kelvin - 273.15

    print(f"max: {to_degrees(max)}, min: {to_degrees(min)}, mean: {to_degrees(mean)}, value: {to_degrees(value)}")